# Daily Stock Prices
##### Purpose: Pull daily stock data using LSEG API for specified rics 
##### Input: List if RICs 
##### Output: Daily stock data from start to end date 
##### For information on the LSEG Eikon API: 
#####         --General: https://developers.lseg.com/en/api-catalog/eikon/eikon-data-api
#####         --API Key: https://developers.lseg.com/en/article-catalog/article/consume-real-time-data-with-refinitiv-data-platform
##### Contact: brds@hbs.edu


## Imports

In [1]:
## Import 
import eikon as ek # Eikon Python wrapper package
import pandas as pd 
import datetime 
import sys
import time
import logging
import IPython

ek.set_app_key('INSERT_API_KEY_HERE') #Set to your API Key

## Specify Rics

In [7]:
# Import Ric List
ric_list = pd.read_excel("Data/Rics_List.xlsx")

# Shorten list for demo
ric_list = ric_list.head(100)

display(ric_list.head(10))


RIC
0  POOL.OQ
1  CHRW.OQ
2    AJG.N
3    CNP.N
4   AMCR.N
5     WM.N
6     BA.N
7   FOX.OQ
8     WY.N
9    MCD.N

## Parameters

In [8]:
## Set Dates
set_start_date = '2024-07-01'
set_end_date = '2024-08-01'

## Chunk Size (set how many RICs to pull at once, based off how many days being pulled -- this helps respect API limits)
chunk_size = 25  # Set Chunk size to respect API limits


## Pull down data

In [9]:
# Set Loop variables
firstloop = 1
last = len(ric_list)

# Loop thru all rics
for start in range(0, last, chunk_size):
    end = start + chunk_size

    try: 
        # Pull Data 
        data_chunk = ek.get_timeseries(ric_list['RIC'].iloc[start:end].tolist(), 
                                start_date = set_start_date,
                                end_date = set_end_date,
                                fields=['CLOSE'], 
                                interval='daily')
        
        # Transpose Data
        data_chunk_t = data_chunk.transpose()
        
        # Append Data just pulled to data pulled in previous loosps
        if firstloop == 1:
            data_t = data_chunk_t
            firstloop = 0 
        else:
            data_t = pd.concat([data_t, data_chunk_t])
        
        print("Pulled: ", end)
        
        
    except: 
            print("Error with rows: ", start, " to ", end)
    time.sleep(15) # Respect API limits



Pulled:  25
Pulled:  50
Pulled:  75
Pulled:  100


## View and Export Data

In [13]:
data_final = data_t.transpose()
display(data_final.head(10))

CLOSE       POOL.OQ  CHRW.OQ   AJG.N  CNP.N  AMCR.N    WM.N    BA.N  FOX.OQ  \
Date                                                                          
2024-07-01   307.69    86.81  259.22  30.55     9.6  209.36   186.7    31.8   
2024-07-02    301.4    86.15   262.0  30.46    9.58  209.47  185.42    32.3   
2024-07-03   301.09    87.51  262.63  30.33    9.62   210.2  184.31   32.44   
2024-07-05   299.78    86.49  264.67  30.49     9.5  210.33  184.83   32.16   
2024-07-08   304.48    86.62  264.54  30.32    9.63  210.62  185.84   32.14   
2024-07-09   296.17    86.55  263.93  29.63    9.56  210.02  183.24   32.76   
2024-07-10   298.87    85.44   267.8  29.59    9.75  211.18  183.73   32.93   
2024-07-11   317.36    86.37  270.12  29.68    9.81  211.77  183.91   33.25   
2024-07-12    328.4    86.76  271.05   30.3    9.91  213.75  182.31   33.35   
2024-07-15   327.04    87.03  270.46  28.28    9.84   216.0  179.11   33.55   

CLOSE        WY.N   MCD.N  ...  COO.OQ  CMG.N  PCAR.OQ  GNRC.N  CHTR.OQ  \
Date                       ...                                            
2024-07-01  27.56  249.99  ...   86.39  61.81   101.84  133.16   296.02   
2024-07-02  27.48  247.79  ...   86.51   61.4   100.91   137.3    303.5   
2024-07-03  27.05   250.0  ...   85.51  61.58    102.0  138.03   301.25   
2024-07-05  27.45  251.09  ...    85.9  62.75   100.22   140.5   301.79   
2024-07-08  27.59  247.85  ...   85.65  59.51   101.28  142.19   290.56   
2024-07-09  27.57  245.82  ...   85.68  57.48   101.34  142.85   293.91   
2024-07-10  27.58  250.49  ...   85.55   58.3    101.5  142.12   293.95   
2024-07-11  28.96   254.8  ...   88.58  57.55   103.23  148.45   304.14   
2024-07-12  29.28   253.9  ...   90.81  57.67   103.11  153.94   311.04   
2024-07-15  29.04  251.53  ...   88.67  56.61   104.55  154.82   324.59   

CLOSE        PWR.N  CCL.N    PM.N   SNA.N  UAL.OQ  
Date                                               
2024-07-01  247.61  17.71  101.25   256.9   48.33  
2024-07-02  247.78  17.56  101.64  257.06    48.2  
2024-07-03   254.6  17.45  101.42  256.89   48.51  
2024-07-05  252.53  17.21  102.12  255.35    47.0  
2024-07-08  254.66  17.56  102.59  257.41   46.82  
2024-07-09  254.79   17.9  101.67  256.11    47.2  
2024-07-10  257.87  17.74  102.76   261.4   47.15  
2024-07-11  264.14  18.12   105.0   267.9   45.64  
2024-07-12  268.81  18.23  105.95  269.63   44.72  
2024-07-15  257.24  18.32  105.07  270.84   44.32  

[10 rows x 100 columns]

In [12]:
data_final.to_excel('Data/Output_DailyStockPrices.xlsx')
